In [1]:
import pandas as pd
import json

# Load the CPV data (this is your JSON file that contains the raw CPV codes)
with open(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\CPVs\CPV.json", "r", encoding="utf-8") as f:
    cpv_data = json.load(f)

# Flatten the CPV codes into a DataFrame for easier handling
cpv_list = []
for line in cpv_data:
    codes_and_descriptions = line.split("031")  # Split based on CPV hierarchical coding
    for item in codes_and_descriptions:
        if item.strip():
            code_description = item.strip().split(" ", 1)  # Split code and description
            if len(code_description) == 2:
                code, description = code_description
                cpv_list.append([code.strip(), description.strip()])

# Create DataFrame
cpv_df = pd.DataFrame(cpv_list, columns=["CPV Code", "Description"])

# Optionally, drop duplicates if the same code appears multiple times
cpv_df = cpv_df.drop_duplicates(subset=["CPV Code"])

# Save the cleaned CPV data
cpv_df.to_csv("cleaned_cpv_data.csv", index=False)

print("Cleaned CPV data saved to 'cleaned_cpv_data.csv'")


Cleaned CPV data saved to 'cleaned_cpv_data.csv'


In [4]:
import re
import pandas as pd

# Function to clean the CPV data
def clean_cpv_data(cpv_df):
    cleaned_cpv_list = []
    
    for index, row in cpv_df.iterrows():
        text = row['CPV Code']
        
        # Split the code from the description. Assuming code ends with 8 digits
        match = re.match(r'(\d{8})(.*)', text)  # Match 8-digit code followed by the description
        if match:
            code = match.group(1).strip()
            description = match.group(2).strip()
            cleaned_cpv_list.append([code, description])
        else:
            # If no match, it means the row is malformed, handle accordingly
            print(f"Malformed row: {text}")

    # Create a cleaned DataFrame
    cleaned_cpv_df = pd.DataFrame(cleaned_cpv_list, columns=['CPV Code', 'Description'])
    
    # Drop duplicates if any
    cleaned_cpv_df = cleaned_cpv_df.drop_duplicates(subset=['CPV Code'])
    
    return cleaned_cpv_df

# Clean the CPV data
cleaned_cpv_df = clean_cpv_data(cpv_df)

# Display the cleaned data
cleaned_cpv_df.head()


Malformed row: 00000Landbruk
Malformed row: 10000Produkter
Malformed row: 13000Sukkerbeter,
Malformed row: 14000Halm
Malformed row: 15000Ubearbeidede
Malformed row: 15100Ubearbeidede
Malformed row: 16000Naturgummi
Malformed row: 16300Lateks
Malformed row: 17000Planter
Malformed row: 17100Planter
Malformed row: 17110Planter
Malformed row: 17120Planter
Malformed row: 17130Planter
Malformed row: 17140Planter
Malformed row: 17200Plantefrø
Malformed row: 20000Landbruks-
Malformed row: 21000Hagebruks
Malformed row: 21100Levende
Malformed row: 21200Avskårne
Malformed row: 30000Avlinger
Malformed row: 31000Planter
Malformed row: 32000Krydderier,
Malformed row: 40000Dyreprodukter
Malformed row: 42000Animalske
Malformed row: 42100Naturlig
Malformed row: 42300Spiselige
Malformed row: 43000Produkter
Malformed row: 44000Landbruksvarer03200000Korn,
Malformed row: 0000Vedlikehold
Malformed row: 1000Vedlikehold
Malformed row: 1400Vedlikehold
Malformed row: 2000Vedlikehold
Malformed row: 2100Vedlikehol

,CPV Code,Description
0,03000000,"Landbruk,"
1,03200000,"Korn,"
2,03210000,Korn
3,03211000,Korn03211100Hvete03211110Durumhvete03211120Vanlig
4,03211100,Hvete03211110Durumhvete03211120Vanlig


In [11]:
import pandas as pd
import re

# Load the CPV data from the CSV
cpv_df = pd.read_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\CPVs\cleaned_cpv_data.csv")

# Function to clean and split CPV codes and descriptions
def clean_and_split_cpv(cpv_df):
    cpv_data = []
    
    for index, row in cpv_df.iterrows():
        text = row['CPV Code']
        description = row['Description']
        
        # Remove any unwanted quotation marks and extra spaces
        text = text.replace('"', '').strip()
        description = description.replace('"', '').strip()
        
        # Remove any CPV codes embedded in the description using regex
        # This regex will match and remove any CPV code embedded inside the description
        description = re.sub(r'\d{8}', '', description).strip()
        
        # Regex to match the CPV code (8 digits) followed by description
        codes_and_descriptions = re.split(r'(\d{8})', text)
        
        # Clean up the resulting parts and make sure each code is properly separated
        for i in range(1, len(codes_and_descriptions), 2):  # Processing every second item (the codes)
            cpv_code = codes_and_descriptions[i].strip()
            if i + 1 < len(codes_and_descriptions):  # If a description follows the code
                cpv_name = codes_and_descriptions[i+1].strip()
            else:
                cpv_name = "No description"  # Handle case where description is missing
                
            # Check if the CPV code already exists in the list
            existing_row = next((item for item in cpv_data if item[0] == cpv_code), None)
            
            if existing_row:
                # Append additional descriptions to the existing CPV code
                existing_row[2] += " | " + description
            else:
                # Add a new row if CPV code doesn't exist
                cpv_data.append([cpv_code, cpv_name, description])

    # Create a structured DataFrame
    cleaned_cpv_df = pd.DataFrame(cpv_data, columns=['CPV Code', 'Name', 'Description'])
    
    # Drop duplicates if any
    cleaned_cpv_df = cleaned_cpv_df.drop_duplicates(subset=['CPV Code'])
    
    return cleaned_cpv_df

# Clean the CPV data and split into code, name, and description
cleaned_cpv_df = clean_and_split_cpv(cpv_df)

# Display the cleaned and organized data
print(cleaned_cpv_df.head())

# Optionally, save the cleaned data to a new CSV
cleaned_cpv_df.to_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\CPVs\final_cleaned_cpv_data.csv", index=False)


   CPV Code       Name                                        Description
0  03000000  Landbruk,  gårdsbruk, fiske, skogsbruk og relaterte produ...
1  03200000      Korn,  poteter, grønnsaker, frukt og nøtterKorn og po...
2  03210000       Korn  og poteterKornHveteDurumhveteVanlig hveteMaisR...
3  03211000       Korn   hveteMaisRisBygg (korn)RugHavreMaltKornprodukter
4  03211100      Hvete  hveteMaisRisBygg (korn)RugHavreMaltKornprodukt...
